In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import joblib

In [3]:
data = pd.read_csv('https://raw.githubusercontent.com/elsamaulida5/machine-learning/main/cleaned%20data/laundry.csv')
data

,ID Laundry,Name,Fulladdress,Categories,phone,Review Count,Average Rating,Review URL,Google Maps URL,Latitude,Longitude,Website,Opening Hours,Featured Image,Layanan
0,1,Laundryklin Sudirman ( Laundry Antar Jemput Ba...,"Jl. Jend. Sudirman No.486, Ciroyom, Kec. Andir...",Layanan Binatu,+62 851-7335-8871,151,4.8,https://search.google.com/local/reviews?placei...,https://www.google.com/maps?cid=94604019837086...,-6.918552,107.586449,NaN,"Sabtu: [07.00-21.00], Minggu: [07.00-21.00], S...",https://lh5.googleusercontent.com/p/AF1QipMUOc...,"just wash, dry cleaning, iron wash"
1,2,The Laundry Room,"Jl. Ciroyom No.95/77, Dungus Cariang, Kec. And...",Layanan Binatu,+62 22 20562320,3,5.0,https://search.google.com/local/reviews?placei...,https://www.google.com/maps?cid=11162829296077...,-6.909928,107.580838,https://www.instagram.com/the.laundry.room.ban...,"Sabtu: [Buka 24 jam], Minggu: [Buka 24 jam], S...",https://lh5.googleusercontent.com/p/AF1QipOt-O...,"just wash, dry cleaning, iron wash"
2,3,Nadhif Laundry ciroyom bandung,"Gg. Sastra No.272/78, Dungus Cariang, Kec. And...",Binatu,+62 22 6036296,6,4.7,https://search.google.com/local/reviews?placei...,https://www.google.com/maps?cid=16206958779682...,-6.915349,107.585807,NaN,"Sabtu: [08.00-20.00], Minggu: [08.00-20.00], S...",https://lh5.googleusercontent.com/p/AF1QipPN10...,"just wash, iron wash"
3,4,Laundry Club Rajawali,"Jl. Rajawali Barat No.25, Garuda, Kec. Andir, ...",Layanan Binatu,+62 851-7324-7944,33,4.9,https://search.google.com/local/reviews?placei...,https://www.google.com/maps?cid=10211050715543...,-6.911077,107.570868,https://bit.ly/customer_laundryclub_rajawali,"Sabtu: [07.00-20.00], Minggu: [09.00-17.00], S...",https://lh5.googleusercontent.com/p/AF1QipMERU...,"just wash, dry cleaning, iron wash"
4,5,EZpress Home Laundry,"Jl. Durman No.4, Kb. Jeruk, Kec. Andir, Kota B...",Layanan Binatu,+62 823-2020-9878,50,4.2,https://search.google.com/local/reviews?placei...,https://www.google.com/maps?cid=89067263696846...,-6.919917,107.599841,NaN,"Sabtu: [07.00-18.00], Minggu: [Tutup], Senin: ...",https://lh5.googleusercontent.com/p/AF1QipOy4x...,"just wash, dry cleaning, iron wash"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1221,1222,Vanili Laundry,"Jl. Vanili Raya No.99, Mangga, Kec. Medan Tunt...",Layanan Binatu,NaN,1,5.0,https://search.google.com/local/reviews?placei...,https://www.google.com/maps?cid=13474658465171...,3.508912,98.632010,0,NaN,https://streetviewpixels-pa.googleapis.com/v1/...,"just wash, dry cleaning, iron wash"
1222,1223,Laundry The Daily Wash dr.mansur,"Jl. Dr. Mansyur No.27, Padang Bulan Selayang I...",Binatu,NaN,16,4.3,https://search.google.com/local/reviews?placei...,https://www.google.com/maps?cid=91915042862606...,3.567332,98.651193,0,NaN,https://lh5.googleusercontent.com/p/AF1QipPZbY...,"just wash, iron wash"
1223,1224,Tetanggamu Laundry,"Jl. Prof. H. M. Yamin No.17b, Perintis, Kec. M...","Binatu, Binatu Cucian Kering",+62 812-6511-5252,71,5.0,https://search.google.com/local/reviews?placei...,https://www.google.com/maps?cid=17346466435953...,3.594387,98.680243,0,"Minggu: [08.30-19.00], Senin: [08.30-19.00], S...",https://lh5.googleusercontent.com/p/AF1QipN-8x...,"just wash, dry cleaning, iron wash"
1224,1225,Mutiara Laundry Express Powered By DEXTER,"Jl. Arief Rahman Hakim No.136, RT.02, Ps. Mera...",Binatu,NaN,9,4.6,https://search.google.com/local/reviews?placei...,https://www.google.com/maps?cid=33600404797552...,3.571973,98.703399,0,"Minggu: [09.00-22.00], Senin: [09.00-22.00], S...",https://lh5.googleusercontent.com/p/AF1QipMi-K...,"just wash, iron wash"


In [4]:
type(data)

pandas.core.frame.DataFrame

In [5]:
# Preprocessing teks (contoh sederhana: lowercase)
data['Name'] = data['Name'].str.lower()
data['Fulladdress'] = data['Fulladdress'].str.lower()
data['Categories'] = data['Categories'].str.lower()
data['Layanan'] = data['Layanan'].str.lower()


In [6]:


# Gabungkan kolom teks yang relevan untuk pencarian
data['Description'] = data['Name'] + ' ' + data['Fulladdress'] + ' ' + data['Categories'] + ' ' + data['Layanan']
data.isna().sum()

ID Laundry           0
Name                 0
Fulladdress          2
Categories           0
phone              131
Review Count         0
Average Rating       0
Review URL          24
Google Maps URL      0
Latitude             0
Longitude            0
Website            760
Opening Hours      102
Featured Image       8
Layanan              3
Description          5
dtype: int64

In [7]:
# Mengganti nilai np.nan dengan string kosong
data['Description'].fillna('', inplace=True)
data.isna().sum()

ID Laundry           0
Name                 0
Fulladdress          2
Categories           0
phone              131
Review Count         0
Average Rating       0
Review URL          24
Google Maps URL      0
Latitude             0
Longitude            0
Website            760
Opening Hours      102
Featured Image       8
Layanan              3
Description          0
dtype: int64

In [8]:
# Inisialisasi TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
# Fit dan transform deskripsi menjadi matriks TF-IDF
tfidf_matrix = tfidf_vectorizer.fit_transform(data['Description'])

# Simpan TF-IDF Vectorizer menggunakan joblib
joblib.dump(tfidf_vectorizer, 'tfidf_vectorizer.pkl')

['tfidf_vectorizer.pkl']

In [9]:
# Fungsi pencarian berbasis teks
def cari_tempat_laundry(keyword, data, tfidf_matrix, tfidf_vectorizer):
    # Transform kata kunci menjadi vektor TF-IDF
    keyword_vec = tfidf_vectorizer.transform([keyword.lower()])

    # Hitung similaritas kosinus antara keyword_vec dan semua dokumen
    similarities = cosine_similarity(keyword_vec, tfidf_matrix).flatten()

    # Dapatkan indeks urutan similaritas tertinggi
    index_sorted = similarities.argsort()[::-1]

    # Ambil data tempat laundry yang paling relevan
    hasil_pencarian = data.iloc[index_sorted]

    return hasil_pencarian


In [10]:
# Contoh pencarian dengan keyword 'laundry harga terjangkau'
keyword = 'bandung'
hasil = cari_tempat_laundry(keyword, data, tfidf_matrix, tfidf_vectorizer)
print(hasil[['Name',	'Fulladdress',	'phone', 'Average Rating',	'Google Maps URL',	'Website',	'Opening Hours',	'Featured Image',	'Layanan']])

                                                  Name  \
243                             laundry kiloan bandung   
240       amala express laundry bandung & wet cleaning   
152  fruits digital laundry bandung (laundry gratis...   
54                                     raundry laundry   
212                            laundry express bandung   
..                                                 ...   
690          aqualis laundry & green dry clean - tebet   
691                                  freshclin laundry   
692    aqualis laundry & green dry clean - tebet barat   
693             kleen laundry & general cleaning tebet   
685                  quick & clean laundry & dry clean   

                                           Fulladdress              phone  \
243  jl. aceh no.22, babakan ciamis, kec. sumur ban...  +62 812-9072-4141   
240  jl. merdeka no.25 - 27, babakan ciamis, kec. s...  +62 819-0819-6626   
152  jl. cijambe no.32, pasir endah, kec. ujung ber...  +62 877-2873-362